# Test of SSD Lite MobileDet

## Setup

In [8]:
import os
import time
import numpy as np
from tflite_runtime.interpreter import Interpreter
from tflite_runtime.interpreter import load_delegate
from PIL import Image, ImageDraw, ImageFont
%matplotlib inline

model_path = '/home/mathias/Documents/experts_in_teams_proj/vision/tf_object_detection/trained_models/ssdlite_mobiledet_fence_hole.tflite'

def xywh2xyxy(x:np.array):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    # y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y = np.copy(x)
    y[0] = x[0] - x[2] / 2  # top left x
    y[1] = x[1] - x[3] / 2  # top left y
    y[2] = x[0] + x[2] / 2  # bottom right x
    y[3] = x[1] + x[3] / 2  # bottom right y
    return y

## Testing

### Simulation

In [27]:
%rm -rf ../utils/map/input/ground-truth
%rm -rf ../utils/map/input/images-optional
%rm -rf ../utils/map/input/detection-results
%rm -rf ../utils/map/output

%mkdir ../utils/map/input/ground-truth
%mkdir ../utils/map/input/images-optional
%mkdir ../utils/map/input/detection-results

In [28]:
# Creates tflite interpreter
interpreter = Interpreter(model_path) 
print(interpreter)
interpreter.allocate_tensors()
interpreter.invoke() # warmup
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
width = input_details[0]['shape'][2]
height = input_details[0]['shape'][1]

def run_inference(interpreter, image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    boxes = interpreter.get_tensor(output_details[0]['index'])[0]
    classes = interpreter.get_tensor(output_details[1]['index'])[0]
    scores = interpreter.get_tensor(output_details[2]['index'])[0]
    # num_detections = interpreter.get_tensor(output_details[3]['index'])[0]
    return boxes, classes, scores

times = list()

test_path = '/home/mathias/Documents/experts_in_teams_proj/vision/yolov5/data/fence_test_sim/test/images'
test_image_paths = sorted([f for f in os.listdir(test_path)]) 
for idx, image_path in enumerate(test_image_paths):
    image = Image.open(os.path.join(test_path, image_path))
    image_width, image_height = image.size
    draw = ImageDraw.Draw(image)
    resized_image = image.resize((width, height))
    np_image = np.asarray(resized_image)
    input_tensor = np.expand_dims(np_image, axis=0)
    
    # Run inference
    model_start = time.time()
    boxes, classes, scores = run_inference(interpreter, input_tensor)
    model_end = time.time() - model_start
    times.append(model_end)
    
    # Draw results on image
    colors = {0:(255, 0, 0)}
    labels = {0:'Hole'}

    
    with open(f'../utils/map/input/detection-results/{idx:04d}.txt', 'w') as txt_file:
    
        for i in range(len(boxes)):
            if scores[i] > 0.5:
                ymin = max(0, boxes[i][0]) # int(max(1, (boxes[i][0] * image_height)))
                xmin = max(0, boxes[i][1]) # int(max(1, (boxes[i][1] * image_width)))
                ymax = min(1.0, boxes[i][2]) # int(min(image_height, (boxes[i][2] * image_height)))
                xmax = min(1.0, boxes[i][3]) # int(min(image_width, (boxes[i][3] * image_width)))
                
                print(f'hole {scores[i]} {xmin} {ymin} {xmax} {ymax}', file=txt_file)
            
                # draw.rectangle((xmin, ymin, xmax, ymax), width=12, outline=colors[int(classes[i])])
                # draw.rectangle((xmin, ymin, xmax, ymin-10), fill=colors[int(classes[i])])
                # text = labels[int(classes[i])] + ' ' + str(scores[i]*100) + '%'
                # draw.text((xmin+2, ymin-10), text, fill=(0,0,0), width=2, font=ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 28))
    
    # display(image)
    # image.save(f'detection-results/{idx:04d}.png')
    
times = np.array(times)
print(f'Took average pr images: {np.mean(times)} s')

Took average pr images: 0.11851510763168335 s


In [29]:
# Load labels into ground-truth folder
gt = '/home/mathias/Documents/experts_in_teams_proj/vision/yolov5/data/fence_test_sim/test/labels'
gt_files = sorted([f for f in os.listdir(gt)])
for i, f in enumerate(gt_files):
    with open(os.path.join(gt, f), 'r') as txt_file:
        lines = txt_file.readlines()
    with open(f'/home/mathias/Documents/experts_in_teams_proj/vision/utils/map/input/ground-truth/{i:04d}.txt', 'w') as txt_file:
        if len(lines) == 0:
                # print('', file=txt_file)
                continue
        else:
            for line in lines:
                line = line.replace('\n', '').split(' ')
                bbox = xywh2xyxy(np.array([
                    float(line[1]), float(line[2]), float(line[3]), float(line[4])
                ]))
                print(f'hole {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}', file=txt_file)

In [30]:
!python ../utils/map/scripts/extra/intersect-gt-and-dr.py

total ground-truth files: 600
total detection-results files: 600

No backup required for /home/mathias/Documents/experts_in_teams_proj/vision/utils/map/input/ground-truth
No backup required for /home/mathias/Documents/experts_in_teams_proj/vision/utils/map/input/detection-results
total intersected files: 600
Intersection completed!


In [31]:
!python ../utils/map/main.py

56.70% = hole AP 
mAP = 56.70%
Figure(640x480)


In [32]:
%rm -rf ../assets/map_mobiledet_sim 
%mkdir ../assets/map_mobiledet_sim
%cp ../utils/map/output/output.txt ../assets/map_mobiledet_sim/output.txt
%cp ../utils/map/output/detection-results-info.png ../assets/map_mobiledet_sim/detection-results-info.png
%cp ../utils/map/output/ground-truth-info.png ../assets/map_mobiledet_sim/ground-truth-info.png
%cp ../utils/map/output/lamr.png ../assets/map_mobiledet_sim/lamr.png
%cp ../utils/map/output/mAP.png ../assets/map_mobiledet_sim/mAP.png
%cp ../utils/map/output/classes/hole.png ../assets/map_mobiledet_sim/hole.png

Results:

| mAP@0.5 |
|:-------:|
| ![](../assets/map_mobiledet_sim/mAP.png) |

| Log-average miss rate |
|:---------------------:|
| ![](../assets/map_mobiledet_sim/lamr.png) |

| Average precision |
|:-----------------:|
| ![](../assets/map_mobiledet_sim/hole.png) |

| Detection results info |
|:----------------------:|
| ![](../assets/map_mobiledet_sim/detection-results-info.png) |

| Ground truth info |
|:-----------------:|
| ![](../assets/map_mobiledet_sim/ground-truth-info.png) |


### Real world

In [33]:
%rm -rf ../utils/map/input/ground-truth
%rm -rf ../utils/map/input/images-optional
%rm -rf ../utils/map/input/detection-results
%rm -rf ../utils/map/output

%mkdir ../utils/map/input/ground-truth
%mkdir ../utils/map/input/images-optional
%mkdir ../utils/map/input/detection-results

In [34]:
# Creates tflite interpreter
interpreter = Interpreter(model_path) 
print(interpreter)
interpreter.allocate_tensors()
interpreter.invoke() # warmup
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
width = input_details[0]['shape'][2]
height = input_details[0]['shape'][1]

def run_inference(interpreter, image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    boxes = interpreter.get_tensor(output_details[0]['index'])[0]
    classes = interpreter.get_tensor(output_details[1]['index'])[0]
    scores = interpreter.get_tensor(output_details[2]['index'])[0]
    # num_detections = interpreter.get_tensor(output_details[3]['index'])[0]
    return boxes, classes, scores

times = list()

test_path = '/home/mathias/Documents/experts_in_teams_proj/vision/yolov5/data/fence_test_real/test/images'
test_image_paths = sorted([f for f in os.listdir(test_path)]) 
for idx, image_path in enumerate(test_image_paths):
    image = Image.open(os.path.join(test_path, image_path))
    image_width, image_height = image.size
    draw = ImageDraw.Draw(image)
    resized_image = image.resize((width, height))
    np_image = np.asarray(resized_image)
    input_tensor = np.expand_dims(np_image, axis=0)
    
    # Run inference
    model_start = time.time()
    boxes, classes, scores = run_inference(interpreter, input_tensor)
    model_end = time.time() - model_start
    times.append(model_end)
    
    # Draw results on image
    colors = {0:(255, 0, 0)}
    labels = {0:'Hole'}

    
    with open(f'../utils/map/input/detection-results/{idx:04d}.txt', 'w') as txt_file:
    
        for i in range(len(boxes)):
            if scores[i] > 0.5:
                ymin = max(0, boxes[i][0]) # int(max(1, (boxes[i][0] * image_height)))
                xmin = max(0, boxes[i][1]) # int(max(1, (boxes[i][1] * image_width)))
                ymax = min(1.0, boxes[i][2]) # int(min(image_height, (boxes[i][2] * image_height)))
                xmax = min(1.0, boxes[i][3]) # int(min(image_width, (boxes[i][3] * image_width)))
                
                print(f'hole {scores[i]} {xmin} {ymin} {xmax} {ymax}', file=txt_file)
            
                # draw.rectangle((xmin, ymin, xmax, ymax), width=12, outline=colors[int(classes[i])])
                # draw.rectangle((xmin, ymin, xmax, ymin-10), fill=colors[int(classes[i])])
                # text = labels[int(classes[i])] + ' ' + str(scores[i]*100) + '%'
                # draw.text((xmin+2, ymin-10), text, fill=(0,0,0), width=2, font=ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 28))
    
    # display(image)
    # image.save(f'detection-results/{idx:04d}.png')
    
times = np.array(times)
print(f'Took average pr images: {np.mean(times)} s')

Took average pr images: 0.11574494558880587 s


In [35]:
# Load labels into ground-truth folder
gt = '/home/mathias/Documents/experts_in_teams_proj/vision/yolov5/data/fence_test_real/test/labels'
gt_files = sorted([f for f in os.listdir(gt)])
for i, f in enumerate(gt_files):
    with open(os.path.join(gt, f), 'r') as txt_file:
        lines = txt_file.readlines()
    with open(f'/home/mathias/Documents/experts_in_teams_proj/vision/utils/map/input/ground-truth/{i:04d}.txt', 'w') as txt_file:
        if len(lines) == 0:
                # print('', file=txt_file)
                continue
        else:
            for line in lines:
                line = line.replace('\n', '').split(' ')
                bbox = xywh2xyxy(np.array([
                    float(line[1]), float(line[2]), float(line[3]), float(line[4])
                ]))
                print(f'hole {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}', file=txt_file)

In [36]:
!python ../utils/map/scripts/extra/intersect-gt-and-dr.py

total ground-truth files: 213
total detection-results files: 213

No backup required for /home/mathias/Documents/experts_in_teams_proj/vision/utils/map/input/ground-truth
No backup required for /home/mathias/Documents/experts_in_teams_proj/vision/utils/map/input/detection-results
total intersected files: 213
Intersection completed!


In [37]:
!python ../utils/map/main.py

37.77% = hole AP 
mAP = 37.77%
Figure(640x480)


In [38]:
%rm -rf ../assets/map_mobiledet_real
%mkdir ../assets/map_mobiledet_real
%cp ../utils/map/output/output.txt ../assets/map_mobiledet_real/output.txt
%cp ../utils/map/output/detection-results-info.png ../assets/map_mobiledet_real/detection-results-info.png
%cp ../utils/map/output/ground-truth-info.png ../assets/map_mobiledet_real/ground-truth-info.png
%cp ../utils/map/output/lamr.png ../assets/map_mobiledet_real/lamr.png
%cp ../utils/map/output/mAP.png ../assets/map_mobiledet_real/mAP.png
%cp ../utils/map/output/classes/hole.png ../assets/map_mobiledet_real/hole.png

Results:

| mAP@0.5 |
|:-------:|
| ![](../assets/map_mobiledet_real/mAP.png) |

| Log-average miss rate |
|:---------------------:|
| ![](../assets/map_mobiledet_real/lamr.png) |

| Average precision |
|:-----------------:|
| ![](../assets/map_mobiledet_real/hole.png) |

| Detection results info |
|:----------------------:|
| ![](../assets/map_mobiledet_real/detection-results-info.png) |

| Ground truth info |
|:-----------------:|
| ![](../assets/map_mobiledet_real/ground-truth-info.png) |
